In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import chess.pgn
import numpy as np
from IPython.display import display, clear_output
import torch
from fastai2.data import *
from fastai2.basics import *
from fastai2.callback.all import *

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
def get_bitboard(board):
    '''
    params
    ------
    board : chess.pgn board object
        board to get state from
    returns
    -------
    bitboard representation of the state of the game
    64 * 6 + 5 dim binary numpy vector
    64 squares, 6 pieces, '1' indicates the piece is at a square
    5 extra dimensions for castling rights queenside/kingside and whose turn
    '''
    bitboard = np.zeros(64*6*2+5)

    piece_idx = {'p': 0, 'n': 1, 'b': 2, 'r': 3, 'q': 4, 'k': 5}

    for i in range(64):
        if board.piece_at(i):
            color = int(board.piece_at(i).color) + 1
            bitboard[(piece_idx[board.piece_at(i).symbol().lower()] + i * 6) * color] = 1

    bitboard[-1] = int(board.turn)
    bitboard[-2] = int(board.has_kingside_castling_rights(True))
    bitboard[-3] = int(board.has_kingside_castling_rights(False))
    bitboard[-4] = int(board.has_queenside_castling_rights(True))
    bitboard[-5] = int(board.has_queenside_castling_rights(False))

    return bitboard

In [5]:
def get_result(game):
    result = game.headers['Result']
    result = result.split('-')
    if result[0] == '1':
        return 1
    elif result[0] == '0':
        return -1
    else:
        return 0

In [257]:
# Change function to write files with correct names 
def pgn_to_np(path):
    games = open(path)
    bitboards = []
    labels = []
    num_games = 0

#     while True:
    for i in range(8):
        if num_games % 1 == 0:
            clear_output()
#             print(num_games)
        num_games += 1

        game = chess.pgn.read_game(games)

        try:
            result = get_result(game)
            board = game.board()
            for move in game.mainline_moves():
                board.push(move)
                bitboard = get_bitboard(board)

                bitboards.append(bitboard)
                labels.append(result)
        except:
            clear_output()
            print(f"{num_games} games in file")
            break

    bitboards = np.array(bitboards)
    labels = np.array(labels)

#     np.save('data/bitboards.npy', bitboards)
#     np.save('data/labels.npy', labels)

    return bitboards, labels

bitboards, labels = pgn_to_np('data/games.pgn')

# bitboards = np.load('data/bitboards.npy')
# labels = np.load('data/labels.npy')

In [272]:
print(bitboards.shape)
bitboards

(650, 773)


array([[0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 0.]])

In [274]:
print(labels.shape)
labels

(650,)


array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,
        1,  1,  1,  1,  1

In [258]:
data = list(zip(bitboards, labels))

def get_x(d): return d[0]
def get_y(d): return d[1]

splits = RandomSplitter()(data)
tfms = [[get_x], [get_y, Categorize()]]
dsets = Datasets(data, tfms=tfms, splits=splits)

dls = dsets.dataloaders(bs=4, device='cpu', num_workers=0)

In [249]:
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear1 = nn.Linear(773, 128)
        self.linear2 = nn.Linear(128, 3)

    def forward(self, feature_vec):
        hidden1 = self.linear1(feature_vec[0].float()).clamp(min=0)  # ReLU
        output = self.linear2(hidden1)
        return F.log_softmax(output, dim=1)

In [250]:
learn = Learner(
    dls, 
    SimpleModel(), 
    loss_func=CrossEntropyLossFlat(), 
    opt_func=partial(Adam, decouple_wd=True), 
    metrics=accuracy
)

In [251]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
0,nan,None,None,00:00
